In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Matrix for computing forward model

We want to create a function that gets the depths of the geophones and the proposed layer structure of the subsurface to compute the **G** matrix defined as follows:

$$ \bf{d} = \bf{G} \bf{m} $$

where $\bf{d}$ is the travel times vector and $\bf{m}$ is the slowness vector of the layers.

In [11]:
def get_G_matrix(geophones_depth, layer_thickness, n_layers):
    """
    Compute matrix G for VSP forward model
    
    .. math ::
        d = Gm
    
    Where `d` is the travel times vector and `m` is the slowness vector,
    whose elements correspond to the slowness of each layer of the model.
    
    Parameter
    ---------
    geophones_depth : array
        Array of depths of the geophones
    layer_thickness : float
        Thickness of the layers, assuming each layer has the same
        thickness.
    n_layers : int
        Total number of layers of same thickness.
        
    Returns
    -------
    G : numpy.matrix
        Matrix G that performs the VSP forward modelling
    """
    # Check if the layer structure is well defined
    if n_layers * layer_thickness < max(geophones_depth):
        raise ValueError(
            "Layer structure must be as deep as the deepest geophone"
        )
    # Initialize G matrix full of zeros
    n_geophones = len(geophones_depth)
    G = np.matrix(np.zeros((n_geophones, n_layers)))
    # Compute elements of G
    for i, depth in enumerate(geophones_depth):
        # Get the number of the layer where the geophone is located
        layer_of_geophone = int(depth // layer_thickness)
        G[i, :layer_of_geophone] = layer_thickness
        G[i, layer_of_geophone] = depth % layer_thickness
    return G

# Read data from `data_vsp.asc`

In [18]:
geophones_depth, travel_times = np.loadtxt("data_vsp.asc", unpack=True)